In [74]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv # Loads .env file.
import os
import time
import pandas as pd

In [75]:
load_dotenv('auth.env')
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')

In [76]:
client_credentials_manager = SpotifyClientCredentials(
    client_id = SPOTIFY_CLIENT_ID,
    client_secret = SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [96]:
def get_track_information(playlist_ids):
    artists = []
    first_artists = []
    track_names = []
    track_ids = []
    
    for playlist_id in playlist_ids:
        resp = sp.playlist_tracks(playlist_id, fields='items.track.artists.name, items.track.name, items.track.id', limit = 100, offset=0, market=None, additional_types=('track'))
        for item in resp['items']:
            track = item['track']
            id = track['id']
            name = track['name']
            
            all_artists = track['artists']
            artists_names = [artist['name'] for artist in all_artists]
            
            artists.append(artists_names)
            first_artists.append(artists_names[0])
            track_names.append(name)
            track_ids.append(id)
    
    return artists, first_artists, track_names, track_ids

In [85]:
def get_track_audio_features(track_ids):
    audio_features_all = []
    batch_size = 100
    num_batches = (len(track_ids) + batch_size - 1) // batch_size

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(track_ids))
        track_ids_batch = track_ids[start_idx:end_idx]
        
        try:
            audio_features_batch = sp.audio_features(tracks = track_ids_batch)
            audio_features_all += audio_features_batch
        except spotipy.SpotifyException as e:
            if e.http_status == 429:
                retry_after = int(e.headers.get('Retry-After', 10))
                print(f"Encountered rate limit. Waiting for {retry_after} seconds before retrying...")
                time.sleep(retry_after + 1)
                audio_features_batch = sp.audio_features(tracks=track_ids_batch)
                audio_features_all += audio_features_batch
            else:
                raise e
            
    return audio_features_all

In [99]:
def make_df(data, audio_features, mood):

    info_df = pd.DataFrame(data, columns = ['track_ids', 'track_names', 'artists', 'first_artists'])
    features_df = pd.DataFrame(data = audio_features, columns = audio_features[0].keys())

    df = pd.concat([info_df, features_df], axis = 1)
    df.reset_index(drop = True, inplace = True)

    track_ids = data['track_ids']
    track_ids_len = len(track_ids)
    mood_list = [mood] * track_ids_len
    df['mood'] = mood_list
    
    return df

In [97]:
happy_playlist_ids = ['37i9dQZF1DX9XIFQuFvzM4', '37i9dQZF1DX889U0CL85jj', '37i9dQZF1DX8Dc28snyWrn', '37i9dQZF1DWYBO1MoTDhZI', '37i9dQZF1DX4fpCWaHOned', '37i9dQZF1DWSf2RDTDayIx', '37i9dQZF1DXa19sXUAHiO1', '37i9dQZF1DX7KNKjOK0o75', '37i9dQZF1DX2sUQwD7tbmL', '37i9dQZF1DWYzpSJHStHHx', '37i9dQZF1DX1BzILRveYHb', '37i9dQZF1DX6fhMYWIyuww']
happy_artists, happy_first_artists, happy_track_names, happy_track_ids = get_track_information(happy_playlist_ids)

In [91]:
audio_features_happy = get_track_audio_features(happy_track_ids)

In [92]:
print(audio_features_happy)

[{'danceability': 0.527, 'energy': 0.415, 'key': 4, 'loudness': -11.451, 'mode': 0, 'speechiness': 0.122, 'acousticness': 0.457, 'instrumentalness': 1.72e-05, 'liveness': 0.117, 'valence': 0.515, 'tempo': 78.169, 'type': 'audio_features', 'id': '1k1Bqnv2R0uJXQN4u6LKYt', 'uri': 'spotify:track:1k1Bqnv2R0uJXQN4u6LKYt', 'track_href': 'https://api.spotify.com/v1/tracks/1k1Bqnv2R0uJXQN4u6LKYt', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1k1Bqnv2R0uJXQN4u6LKYt', 'duration_ms': 125093, 'time_signature': 4}, {'danceability': 0.768, 'energy': 0.367, 'key': 2, 'loudness': -11.226, 'mode': 1, 'speechiness': 0.0315, 'acousticness': 0.683, 'instrumentalness': 1.79e-05, 'liveness': 0.081, 'valence': 0.532, 'tempo': 103.621, 'type': 'audio_features', 'id': '3zBhihYUHBmGd2bcQIobrF', 'uri': 'spotify:track:3zBhihYUHBmGd2bcQIobrF', 'track_href': 'https://api.spotify.com/v1/tracks/3zBhihYUHBmGd2bcQIobrF', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3zBhihYUHBmGd2bcQIobrF', 'd

In [100]:
happy_data = {
    'track_ids': happy_track_ids,
    'track_names': happy_track_names,
    'first_artists': happy_first_artists,
    'artists': happy_artists,
}

happy_df = make_df(happy_data, audio_features_happy, "happy")
happy_df.to_csv('happy_data.csv', index = False)

In [101]:
sad_playlist_ids = ['37i9dQZF1DWSqBruwoIXkA', '37i9dQZF1DWW2hj3ZtMbuO', '37i9dQZF1DX7gIoKXt0gmx', '37i9dQZF1DWZrBs4FjpxlE', '37i9dQZF1DX59NCqCqJtoH', '37i9dQZF1DWVV27DiNWxkR', '37i9dQZF1DWVrtsSlLKzro', '37i9dQZF1DWZUAeYvs88zc', '37i9dQZF1DWU4lunzhQdRx', '37i9dQZF1DWV1bxlagjEmb', '37i9dQZF1DX9AnYEthXLyU', '37i9dQZF1DX15JKV0q7shD']
sad_artists, sad_first_artists, sad_track_names, sad_track_ids = get_track_information(sad_playlist_ids)

In [102]:
audio_features_sad = get_track_audio_features(sad_track_ids)

In [111]:
sad_data = {
    'track_ids': sad_track_ids,
    'track_names': sad_track_names,
    'first_artists': sad_first_artists,
    'artists': sad_artists,
}

sad_df = make_df(sad_data, audio_features_sad, "sad")
sad_df.to_csv('sad_data.csv', index = False)

In [112]:
combined_df = pd.concat([happy_df, sad_df], ignore_index = True)
combined_df.to_csv('combined_data.csv', index = False)

In [113]:
combined_df.drop_duplicates(subset = 'id', keep = 'first', inplace = True)
combined_df.reset_index(drop = True, inplace = True)

columns_to_drop = ['analysis_url', 'duration_ms', 'id', 'key', 'liveness', 'speechiness', 'time_signature', 'track_href', 'type', 'uri']
cleaned_df = combined_df.drop(columns=columns_to_drop)
cleaned_df.to_csv('cleaned_data.csv', index = False)